In [1]:
import sys
import os
src_dir = os.path.join(os.getcwd(), '../src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

from nn_model import *
from run_nn import *
from train_nn import *
from dataloader import *

import pandas as pd

print(os.getcwd())


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


/Users/isomorphicdude/Library/Mobile Documents/com~apple~CloudDocs/MSc/SB2.2_Statistical_Machine_Learning/Practicals/Assessed/SML_pratical/notebooks


In [2]:
# Load the training data and the test inputs
X_all = pd.read_csv('../data/X_train.csv', index_col = 0, header=[0, 1, 2]) # inputs of the training set
y_all = pd.read_csv('../data/y_train.csv', index_col = 0).squeeze('columns') # outputs of the training set
X_test = pd.read_csv('../data/X_test.csv', index_col = 0, header=[0, 1, 2]) # inputs of the test set

In [3]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA_model = LinearDiscriminantAnalysis(n_components=None)

In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
y_all = LabelEncoder().fit_transform(y_all)

In [11]:
from sklearn.model_selection import train_test_split
# create dataset
X_train, X_val, y_train, y_val = train_test_split(X_all, 
                                                  y_all, 
                                                  test_size=0.2, 
                                                  random_state=42)

# convert to numpy
X_train = X_train.to_numpy()
X_train = LDA_model.fit_transform(X_train, y_train)
X_val = X_val.to_numpy()
X_val = LDA_model.fit_transform(X_val, y_val)

In [12]:
from torch.utils.data import DataLoader

train_set = MusicData(X_train, y_train)
val_set = MusicData(X_val, y_val)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)

In [13]:
mlp = MLP(input_dim=X_train.shape[1], 
          hidden_dims=[32, 16], 
          output_dim=8)

train_loss, test_loss = run_training_loop(
    num_epochs=1000,
    train_loader=train_loader,
    val_loader=val_loader,
    model=mlp,
    optimizer=torch.optim.Adam(mlp.parameters(), lr=0.1),
    loss_function=torch.nn.CrossEntropyLoss(),
    device="cpu",
)

INFO:root:Path not provided, saving model to models
INFO:root:Epoch: 1, train Loss: 1.2537, val Loss: 3.9183, val Accuracy: 0.0885
INFO:root:Saving model...
INFO:root:Epoch: 6, train Loss: 1.0642, val Loss: 4.1310, val Accuracy: 0.0979
INFO:root:Epoch: 11, train Loss: 1.0548, val Loss: 3.6534, val Accuracy: 0.0970
INFO:root:Saving model...
INFO:root:Epoch: 16, train Loss: 1.0632, val Loss: 3.5059, val Accuracy: 0.1508
INFO:root:Saving model...
INFO:root:Epoch: 21, train Loss: 1.0715, val Loss: 3.6622, val Accuracy: 0.1080
INFO:root:Epoch: 26, train Loss: 1.0763, val Loss: 3.3359, val Accuracy: 0.1357
INFO:root:Saving model...
INFO:root:Epoch: 31, train Loss: 1.0482, val Loss: 4.0291, val Accuracy: 0.0869
INFO:root:Epoch: 36, train Loss: 1.0751, val Loss: 2.9834, val Accuracy: 0.1494
INFO:root:Saving model...
INFO:root:Epoch: 41, train Loss: 1.0667, val Loss: 3.4430, val Accuracy: 0.0924
INFO:root:Epoch: 46, train Loss: 1.0511, val Loss: 3.4892, val Accuracy: 0.1431
INFO:root:Epoch: 51,

KeyboardInterrupt: 

In [10]:
from sklearn.metrics import accuracy_score
# Load model state dict
mlp.load_state_dict(torch.load('models/model.pth'))
mlp.eval()

# predict
y_pred = mlp(torch.tensor(X_val).float()).detach().numpy()

accuracy_score(y_val, np.argmax(y_pred, axis=1))

0.1775